## quickmt train vocab

In [ ]:
import datasets
import sentencepiece as spm
from tqdm.auto import tqdm

In [ ]:
src_lang = "ar"
tgt_lang = "en"
src_lang_flores = "arb_Arab"
tgt_lang_flores = "eng_Latn"

dataset = "quickmt/quickmt-train.ar-en"

vocab_size = 20000
character_coverage = 0.9999
num_threads = 6
limit = 10_000_000
counter = 0

In [ ]:
try:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{src_lang_flores}-{tgt_lang_flores}",
    )
except:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{tgt_lang_flores}-{src_lang_flores}",
    )

src = []
ref = []
for i in flores["dev"]:
    src.append(i[f"sentence_{src_lang_flores}"])
    ref.append(i[f"sentence_{tgt_lang_flores}"])

In [ ]:
with open(f"flores-dev.{src_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in src]))
with open(f"flores-dev.{tgt_lang}", "wt") as myfile:
    myfile.write("".join([i + "\n" for i in ref]))

In [ ]:
ds = datasets.load_dataset(dataset, streaming=True)["train"]

with open(f"{src_lang}.txt", "wt") as src_file, open(
    f"{tgt_lang}.txt", "wt"
) as tgt_file:
    for i in tqdm(ds):
        src_file.write(i[src_lang] + "\n")
        tgt_file.write(i[tgt_lang] + "\n")
        counter += 1
        if limit > 0 and counter >= limit:
            break

In [ ]:
!wc -l {src_lang}.txt

In [ ]:
!wc -l {tgt_lang}.txt

In [ ]:
spm.SentencePieceTrainer.train(
    input=f"{src_lang}.txt",
    model_prefix=f"{src_lang}.spm",
    vocab_size=vocab_size,
    character_coverage=character_coverage,
    num_threads=num_threads,
    shuffle_input_sentence=True,
    byte_fallback=True,
)

In [ ]:
spm.SentencePieceTrainer.train(
    input=f"{tgt_lang}.txt",
    model_prefix=f"{tgt_lang}.spm",
    vocab_size=vocab_size,
    character_coverage=character_coverage,
    shuffle_input_sentence=True,
    num_threads=num_threads,
    byte_fallback=True,
)